# Difference-in-Differences (minimal) for time series

This notebook demonstrates a minimal DiD-style estimator using aggregated time series:

- treated series `y`
- control series `control_series`
- intervention date

We estimate the effect as the pre/post change in the treated-control difference, with block bootstrap CIs.


In [ ]:
import sys
from pathlib import Path

# Ensure `src/` is importable when running from repo root
repo_root = Path.cwd()
src_path = repo_root / "src"
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tecore.causal import DataSpec, ImpactConfig, ImpactMethod, run_impact


## 1) Build a small synthetic treated/control example

In [ ]:
dates = pd.date_range("2025-01-01", periods=140, freq="D")
t = np.arange(len(dates))
control = 100 + 0.08 * t + 2.5 * np.sin(2 * np.pi * t / 7)
treated = control.copy()

intervention_date = "2025-04-01"
post = dates >= pd.Timestamp(intervention_date)
treated[post] += 3.5

df = pd.DataFrame({
    "date": dates.strftime("%Y-%m-%d"),
    "y": treated,
    "control_series": control,
})

df.head()

## 2) Run DiD

In [ ]:
spec = DataSpec(
    date_col="date",
    y_col="y",
    x_cols=["control_series"],
    freq="D",
)

cfg = ImpactConfig(
    intervention_date=intervention_date,
    method=ImpactMethod.DID,
    bootstrap_iters=200,
    block_size=7,
    alpha=0.05,
    run_placebo=True,
    n_placebos=25,
)

res = run_impact(df, spec, cfg)
res.summary()

### Plot treated vs control

In [ ]:
dfp = df.copy()
dfp["date"] = pd.to_datetime(dfp["date"])

plt.figure()
plt.plot(dfp["date"], dfp["y"], label="treated")
plt.plot(dfp["date"], dfp["control_series"], label="control")
plt.axvline(pd.Timestamp(intervention_date), linestyle="--")
plt.title("Treated vs control")
plt.legend()
plt.tight_layout()
plt.show()

### Point effect

In [ ]:
eff = res.effect_series.copy()
eff["date"] = pd.to_datetime(eff["date"])

plt.figure()
plt.plot(eff["date"], eff["point_effect"], label="point_effect")
plt.fill_between(eff["date"], eff["point_ci_low"], eff["point_ci_high"], alpha=0.2, label="CI")
plt.axhline(0.0, linestyle="--")
plt.axvline(pd.Timestamp(intervention_date), linestyle="--")
plt.title("DiD point effect")
plt.legend()
plt.tight_layout()
plt.show()

## 3) Save outputs

In [ ]:
out_dir = repo_root / "out" / "notebook_11_did"
out_dir.mkdir(parents=True, exist_ok=True)

eff_out = out_dir / "effect_series.csv"
eff_to_save = res.effect_series.copy()
eff_to_save["date"] = pd.to_datetime(eff_to_save["date"]).dt.strftime("%Y-%m-%d")
eff_to_save.to_csv(eff_out, index=False)

import json
with open(out_dir / "summary.json", "w", encoding="utf-8") as f:
    json.dump(res.summary(), f, indent=2, ensure_ascii=False)

print("Wrote:", out_dir)